In [1]:
to_encode = "weights" #change to "activations" to encode activations
nums_per_line = 8

In [2]:
filename = to_encode + "_input.txt"

f = open(filename, "r")
data = [] # DATALINES: list of lines representing nums_per_line 4-bit numbers

for line in f:
    data.append(line.strip()) # append bitstring without newline character

print(data)

f.close()

['00000000000000000001000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000001000000000000', '00000000000000000000000000000000', '11000000110000000000000000000110', '00000000000000010000000000000000', '00000000110100101101000011100001', '00000000001011000000000000100000', '11010000000000000001000000100000', '00000000000000000000000000000000', '00010000000000100000000000001101', '00000000001011100001000000000000', '00000000000000000000000000000001', '00000000000000000000000000000000', '00000000000100000000000000000000', '00000000000100000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000001110', '00000000000000000001000000000000', '00000000000000110000000000000000', '00110000000000110000000011100000', '00000000000000010000000000

In [3]:
# Load list of huffman strings

nums = []
# PARSED NUMS: separate 4-bit numbers (length of nums is nums_per_line)
for d in data:
    for i in range(len(d)//4):
        nums.append(d[4*i:4*(i+1)])

strings = [] #4-BIT STRINGS

# generate all possible 4-bit strings
for i in range(2):
    string = str(i)
    for j in range(2):
        string += str(j)
        for k in range(2):
            string += str(k)
            for l in range(2):
                strings.append(string + str(l))
            string = string[:len(string)-1]
        string = string[:len(string)-1]
    string = string[:len(string)-1]

freqs = [] #FREQUENCIES

# find the frequencies of the 4-bit strings
for s in strings:
    freqs.append(nums.count(s))

print(strings)
print(freqs)

['0000', '0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111']
[479, 28, 13, 8, 9, 4, 2, 8, 0, 1, 1, 1, 5, 10, 7, 0]


In [4]:
import heapq

class Node:
    def __init__(self, frequency, value, left=None, right=None):
        self.freq = frequency
        self.val = value
        self.left = left
        self.right = right
        
        self.dir = ''
        self.mapping = ''
        
    # redefine less than operator to compare the frequencies of nodes
    def __lt__(self, other_node):
        return self.freq < other_node.freq
    
    def __str__(self):
        return "NODE value: {0} \tfrequency: {1}\nleft: {2} \tright: {3}\n\n".format(self.val, self.freq, self.left, self.right)

encoding_map = {}

# Credit: this code for printing based on "printNodes" method (Geeks for Geeks)
def saveHuffmanCode(node, val=''): 
    # huffman code for current node 
    newVal = val + str(node.dir) 

    # if node is not an edge node 
    # then traverse inside it 
    if(node.left): 
        saveHuffmanCode(node.left, newVal) 
    if(node.right): 
        saveHuffmanCode(node.right, newVal) 

    # if node is edge node then 
    # display its huffman code 
    if(not node.left and not node.right): 
        print(f"{node.val} -> {newVal}")
        
        #comment out if only printing and not saving
        #node.mapping = newVal
        
        encoding_map[node.val] = newVal #store value to encoding mappings
    
    
nodes = [] #hold initial nodes

for s in range(len(strings)):
    heapq.heappush(nodes, Node(freqs[s], strings[s]))

while len(nodes) > 1:
    left = heapq.heappop(nodes)
    right = heapq.heappop(nodes)
    
    left.dir = 0
    right.dir = 1
    
    # combine smallest nodes for new node / parent
    new_node = Node(left.freq + right.freq, left.val + right.val, left, right)
    
    heapq.heappush(nodes, new_node)
    
saveHuffmanCode(nodes[0]) #print mappings, save mappings to nodes (can be used to export huffman tree for decoding)

0101 -> 00000
1100 -> 00001
0100 -> 0001
1101 -> 0010
0110 -> 001100
1001 -> 0011010
1010 -> 00110110
1000 -> 0011011100
1111 -> 0011011101
1011 -> 001101111
1110 -> 00111
0001 -> 010
0010 -> 0110
0011 -> 01110
0111 -> 01111
0000 -> 1


In [5]:
# encode and output
filename = "encoded_" + to_encode + ".txt"
f = open(filename, "w")

nums_written = 0
for n in nums:
    f.write(encoding_map[n])
    
    # break lines when nums_per_line encoded numbers are written
    nums_written += 1
    if(nums_written % nums_per_line == 0):
        f.write("\n")